In [1]:
import json

# function to read the json file with the annotations of id cards
def readAnnotationsFile(jsonFile):
    try:
        with open(jsonFile, "r") as read_file:
            data = json.load(read_file)
        return data
    except FileNotFoundError:
        print(f"No such file or directory exists: '{jsonFile}'")
        return None

# function to get the image name, and the coordinates of id card position in the image
def retrieveImageAttributes(jsonFile):
    
    # data contains the json data read from the json file
    data = readAnnotationsFile(jsonFile)
    
    if(data):
        
        
        imageAttributesList = []                            # list to contain attributes of all images in the json file
        imageMetadata = data["_via_img_metadata"]           # metadata of image in which the coordinates are present  
        imageIdList = data["_via_image_id_list"]            # names of images in the metadata

        for img in imageIdList:

            fileName = imageMetadata[img]['filename']                          # filename to which the ground truth image wil be stored
            regions = imageMetadata[img]['regions']
            all_points_x = regions[1]['shape_attributes']['all_points_x']      # all x coordinates  
            all_points_y = regions[1]['shape_attributes']['all_points_y']      # all y coordinates
            imageAttributesList.append((fileName,all_points_x,all_points_y))   

        return imageAttributesList                                             # list pf tuples containing (filename, x coordinates and y coordinates) for every image
    else:
        return None

In [36]:
import numpy as np
from skimage.draw import polygon
from matplotlib import pyplot as plt

# function to create a groud truth image at the specified path given
def groundTruth(y_len, x_len, row_arr, col_arr, writeFileName):
    img = np.zeros((y_len, x_len), dtype=np.uint8)
    r = np.array(row_arr)
    c = np.array(col_arr)
    rr, cc = polygon(r,c)
    img[rr,cc] = 1
    
    plt.imsave(writeFileName, img)

In [37]:
import os 

def createDirectory(filePath, newDirectory):
    path = os.path.join(filePath, newDirectory)
    os.mkdir(path)
    
    return path

def createMask(filePath, fileName, imgHeight, imgWidth, all_points_y, all_pints_x):

    writeFilePath = os.path.join(filePath, fileName)
    groundTruth(imgHeight, imgWidth, all_points_y, all_pints_x, writeFilePath)

In [43]:
PHOTO_HEIGHT = 4032
PHOTO_WIDTH = 2268

SCAN_HEIGHT = 3507
SCAN_WIDTH = 2480

pathPrefix = "C:\\Users\\rathi\Desktop\\tmask"

maskPathPrefix = "C:\\Users\\rathi\\Desktop\\try"



pathSuffix = ["annotations\\alb_id.json", "annotations\\esp_id.json"]
idClass = ["alb_id", "esp_id"]



typePath = ["photo", "scan_rotated", "scan_upright"]


for fol in typePath:
    maskDir = createDirectory(os.path.join(maskPathPrefix, fol), "mask")
    for i in range(len(pathSuffix)):
        imageAttributeList = retrieveImageAttributes(os.path.join(pathPrefix, fol, pathSuffix[i]))
        idPath = createDirectory(maskDir, idClass[i])

        for img in imageAttributeList:
            if(fol == "photo"):
                createMask(idPath, img[0], PHOTO_HEIGHT, PHOTO_WIDTH, img[2], img[1])
            else:
                createMask(idPath, img[0], SCAN_HEIGHT, SCAN_WIDTH, img[2], img[1])